In [1]:
#Mount google drive
#import requests
#from google.colab import drive
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#Download the source zip file
#!pip install -U -q kaggle
#from google.colab import files
#files.upload()
#!kaggle datasets list
#!kaggle datasets download -d imdevskp/corona-virus-report

#dload lib didn't work!
#!pip install dload
#https://pypi.org/project/dload/
#import dload
#dload.save_unzip('https://www.kaggle.com/imdevskp/corona-virus-report/download', 'drive/My Drive/PythonFiles'

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
#data source, removing unused columns and cleaning-up date format
import pandas as pd
#df = pd.read_csv('/content/corona-virus-report.zip', compression='zip', header=0, sep=',', quotechar='"')
df = pd.read_csv("http://g0mesp1res.dynip.sapo.pt/covid_19_clean_complete.csv")
df=df.drop(labels=None, axis=0, index=None, columns=['Province/State','Lat','Long'], level=None, inplace=False, errors='raise')
df['Date']=pd.to_datetime(df['Date'])
pd.set_option("display.precision", 2)

In [2]:
#Save the unzip data source to My google Drive
#!unzip -uq '/content/corona-virus-report.zip' -d "drive/My Drive/PythonFiles"

unzip:  cannot find or open /content/corona-virus-report.zip, /content/corona-virus-report.zip.zip or /content/corona-virus-report.zip.ZIP.


In [0]:
#aggregation of data in countries which report data per state or region
df = df.groupby(['Date','Country/Region']).agg(sum).reset_index()
df["DRate"] = 100* df.Deaths / df.Confirmed
#df.tail()

#reducing the dataset
list_countries = ['Portugal','Brazil','Spain','Italy','Germany','South Korea','Japan','France','US','United Kingdom','China']
df= df[df['Country/Region'].isin(list_countries)]

In [9]:
#TOTAL data set
df_total = df.groupby(df['Date'])['Deaths','Recovered','Confirmed'].sum().reset_index()
df_total["DRate"] = 100* df_total.Deaths / df_total.Confirmed
#df_total.to_csv(r'drive/My Drive/PythonFiles/df_total.csv', index = False)
#df_total.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [10]:
#TOTAL data set graphics
import plotly.graph_objects as go
fig = go.Figure( go.Scatter(x=df_total.Date, y=df_total.DRate, name='Total'))
fig.update_layout(title_text='Total Death Rate %', xaxis_rangeslider_visible=True, yaxis_type="log")
fig.show()

In [11]:
import plotly.graph_objects as go

fig = go.Figure( go.Scatter(x=df[df['Country/Region'] == 'Portugal'].Date, y=df[df['Country/Region'] == 'Portugal'].Deaths, name='Portugal', stackgroup='one', line=dict(width =2)))
list = ['Brazil','Spain','Italy','Germany','South Korea','Japan','France','US','United Kingdom','China']
for i in list: 
    fig.add_scatter(x=df[df['Country/Region'] == i].Date, y=df[df['Country/Region'] == i].Deaths, name=i, stackgroup='one')
#fig.add_scatter(x=df_total.Date, y=df_total.Deaths, name='Total')
fig.update_layout(title_text='Deaths per country', xaxis_rangeslider_visible=False)
fig.show()

fig = go.Figure( go.Scatter(x=df[df['Country/Region'] == 'Portugal'].Date, y=df[df['Country/Region'] == 'Portugal'].DRate, line_shape='spline', name='Portugal', line=dict(width =5)))
list = ['Brazil','Spain','Italy','Germany','South Korea','Japan','France','US','United Kingdom','China']
for i in list: 
    fig.add_scatter(x=df[df['Country/Region'] == i].Date, y=df[df['Country/Region'] == i].DRate, line_shape='spline', name=i)
fig.add_scatter(x=df_total.Date, y=df_total.DRate, line_shape='spline', name='Total')
fig.update_layout(title_text='DRate % per country', xaxis_rangeslider_visible=True)
fig.show()

In [0]:
#new data set starting on the 10th death, and dropping the date
df1 = df[df.Deaths >9]
df_total1 = df_total.drop(labels=None, axis=0, index=None, columns=['Date'], level=None, inplace=False, errors='raise')
df_total1['DConfirmed'] = df_total1['Confirmed'] - df_total1['Confirmed'].shift(1)
df_total1['DDeaths'] = df_total1['Deaths'] - df_total1['Deaths'].shift(1)

#don't know how to calculate the difference between rows based on the country
#one idea is to reorganized the df1 per country, and remove the 1st row of each country
#df1 = df1.groupby('Country/Region', sort=True['Country/Region']).agg(sum).reset_index()
df1 = df1.sort_values(['Country/Region','Date'])
#new columns for Delta Confirmed and Delta Deaths
df1['DConfirmed'] = df1['Confirmed'] - df1['Confirmed'].shift(1)
df1['DDeaths'] = df1['Deaths'] - df1['Deaths'].shift(1)
df1 = df1.drop(labels=None, axis=0, index=None, columns=['Date'], level=None, inplace=False, errors='raise')

In [13]:
import plotly.graph_objects as go

fig = go.Figure( go.Scatter(y=df1[df1['Country/Region'] == 'Portugal'].Deaths, line_shape='spline', name='Portugal', line=dict(width =5)))
list = ['Brazil','Spain','Italy','Germany','South Korea','Japan','France','US','United Kingdom','China']
for i in list: 
    fig.add_scatter(y=df1[df1['Country/Region'] == i].Deaths, line_shape='spline', name=i)
#fig.add_scatter(y=df_total1.Deaths, line_shape='spline', name='Total')
fig.update_layout(title_text='Deaths per country since 10th death', xaxis_rangeslider_visible=False,yaxis_type="log")
fig.show()


fig = go.Figure( go.Scatter(y=df1[df1['Country/Region'] == 'Portugal'].DRate, line_shape='spline', name='Portugal', line=dict(width =5)))
list = ['Brazil','Spain','Italy','Germany','South Korea','Japan','France','US','United Kingdom','China']
for i in list: 
    fig.add_scatter(y=df1[df1['Country/Region'] == i].DRate, line_shape='spline', name=i)
#fig.add_scatter(y=df_total1.Deaths, line_shape='spline', name='Total')
fig.update_layout(title_text='Death rate per country since 10th death', xaxis_rangeslider_visible=False)
fig.show()


fig = go.Figure( go.Scatter(y=df1[df1['Country/Region'] == 'Portugal'].DDeaths, line_shape='spline', name='Portugal', line=dict(width =5)))
list = ['Brazil','Spain','Italy','Germany','South Korea','Japan','France','US','United Kingdom','China']
for i in list: 
    fig.add_scatter(y=df1[df1['Country/Region'] == i].DDeaths, line_shape='spline', name=i)
#fig.add_scatter(y=df_total1.Deaths, line_shape='spline', name='Total')
fig.update_layout(title_text='New deaths per country since 10th death', xaxis_rangeslider_visible=False,yaxis_type="log")
fig.show()


fig = go.Figure( go.Scatter(x=df1[df1['Country/Region'] == 'Portugal'].Deaths, y=df1[df1['Country/Region'] == 'Portugal'].DDeaths, line_shape='spline', name='Portugal', line=dict(width =5)))
list = ['Brazil','Spain','Italy','Germany','South Korea','Japan','France','US','United Kingdom','China']
for i in list: 
    fig.add_scatter(x=df1[df1['Country/Region'] == i].Deaths, y=df1[df1['Country/Region'] == i].DDeaths, line_shape='spline', name=i)
fig.add_scatter(x=df_total1.Deaths, y=df_total1.DDeaths, line_shape='spline', name='Total')
fig.update_layout(title_text='Deaths per country since 10th death, x=Total, y=New', xaxis_rangeslider_visible=False,xaxis_type="log",yaxis_type="log")
fig.show()


fig = go.Figure( go.Scatter(x=df1[df1['Country/Region'] == 'Portugal'].Confirmed, y=df1[df1['Country/Region'] == 'Portugal'].DConfirmed, line_shape='spline', name='Portugal', line=dict(width =5)))
list = ['Brazil','Spain','Italy','Germany','South Korea','Japan','France','US','United Kingdom','China']
for i in list: 
    fig.add_scatter(x=df1[df1['Country/Region'] == i].Confirmed, y=df1[df1['Country/Region'] == i].DConfirmed, line_shape='spline', name=i)
fig.add_scatter(x=df_total1.Confirmed, y=df_total1.DConfirmed, line_shape='spline', name='Total')
fig.update_layout(title_text='Confirmed per country since 10th death, x=Total, y=New', xaxis_rangeslider_visible=False,xaxis_type="log",yaxis_type="log")
fig.show()


fig = go.Figure( go.Scatter(y=df1[df1['Country/Region'] == 'Portugal'].DConfirmed, line_shape='spline', name='Portugal', line=dict(width =5)))
list = ['Brazil','Spain','Italy','Germany','South Korea','Japan','France','US','United Kingdom','China']
for i in list: 
    fig.add_scatter(y=df1[df1['Country/Region'] == i].DConfirmed, line_shape='spline', name=i)
#fig.add_scatter(y=df_total1.Deaths, line_shape='spline', name='Total')
fig.update_layout(title_text='New cases per country since 10th death', xaxis_rangeslider_visible=False,yaxis_type="log")
fig.show()


